In [1]:
import io
import subprocess
import pickle
import random
import copy

# Logic Symbols
- & is AND
- | is OR
- ~ is NOT
# CNF creation
Start with a list of literals:
- Literal -> ~Variable
- Literal -> Variable

Disjunctions are made from literals or other disjunctions
- Disjunction -> Literal | Disjunction
- Disjunction -> Literal

Conjunctive normal forms (CNF) are made of disjunctions and other CNFs
- CNF -> (Disjunction) & CNF
- CNF -> (Disjunction)




In [2]:
variables = ['a', 'b', 'c', 'd']
number_of_disjunctions_to_add  = 1
number_of_conjunctions_to_add = 1


literals = []
for element in variables:
        literals.append(element)
        literals.append('~'+element)

disjunctions = copy.deepcopy(literals)

for _ in range(number_of_disjunctions_to_add):
    for i in range(len(disjunctions)):
        for literal in literals:
            if disjunctions[i] is not literal:
                disjunctions.append(disjunctions[i]+'|'+literal)

conjunctive_normal_forms = []
for d in disjunctions:
     conjunctive_normal_forms.append('('+str(d)+')')

for _ in range(number_of_conjunctions_to_add):
    for i in range(len(conjunctive_normal_forms)):
        for d in disjunctions:
            if conjunctive_normal_forms[i] is not d:
                conjunctive_normal_forms.append('('+str(d)+')'+'&'+conjunctive_normal_forms[i])

4160


# TPTP Formatting
fof(name,formula_type,statement)
- name is the name of the formula
- formula_type is either axiom or conjecture, for our purposes.
- Axioms are facts
- Conjectures are what we want to prove/disprove


Extra TPTP notes:
- ! is the universal quantifier
- ? is the existential quantifier
- variables are denoted with capital letters
- func(C) is a function with variable C
- => is the implication operator
- & is AND
- | is OR
- ~ is NEGATION
- <=> is EQUIVALENCE
- = is EQUALITY
- != is INEQUALITY

In [ ]:
statements = list()
conclusions = list()
for e in conjunctive_normal_forms:
    statements.append(e)
    conclusions.append(e)

statement_list = list()
conclusion_list = list()


for i in range(len(statements)):
    statement_list.append(["fof(\'"+str(statements[i])+"\',axiom,"+str(statements[i])+").",statements[i]])

for i in range(len(conclusions)):
    conclusion_list.append(["fof(\'"+str(conclusions[i])+"\',conjecture,"+str(conclusions[i])+").",conclusions[i]])

theorum_list = list()

for s in statement_list:
    for c in conclusion_list:
        input = s[0]+' '+c[0]
        plain_text = str(s[1])+".>"+str(c[1])+"."
        theorum_list.append([input,plain_text])

In [ ]:
file_path = "f.tptp"
eprover_path = "/home/anmarch/source/eprover/PROVER/eprover"
found_proofs = list()
unfound_proofs = list()

for i in range(len(theorum_list)):
    with io.open(file_path,'w',encoding='utf-8') as f:
        f.write(str(theorum_list[i][0]))

    result = subprocess.run([eprover_path, "--proof-object", str(file_path)], capture_output=True)
    output = result.stdout.decode()

    if result.returncode == 0:
        #proof found
        found_proofs.append([str(theorum_list[i][0]),result,theorum_list[i][1]])

    elif result.returncode == 1:
        #proof not found
        unfound_proofs.append([str(theorum_list[i][0]),result,theorum_list[i][1]])

    else:
        #something else happened
        print(result)
        raise Exception("Something unexpected occured")

In [ ]:
def format_training_data(found, unfound):
    input = []
    theorum = []
    non_theorum = []
    for sample in found:
        s = sample[2]
        r = "Found"
        proof = sample[1]
        input.append([s,r,proof])
        theorum.append([s,r,proof])

    for sample in unfound:
        s = sample[2]
        r = "Unfound"
        proof = sample[1]
        input.append([s,r,proof])
        non_theorum.append([s,r,proof])
    
    return [input, theorum, non_theorum]

training_data,found,unfound = format_training_data(found_proofs,unfound_proofs)

with open('training_data.pickle','wb') as f:
    pickle.dump(training_data,f)

with open('theorum.pickle','wb') as f:
    pickle.dump(found,f)

with open('non_theorum.pickle','wb') as f:
    pickle.dump(unfound,f)